In [1]:
import pandas as pd
import numpy as np
import csv


c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [8]:

#Cambiare DATASET per cambiare il modello usato
DATASET='USAD'

GT=[]
TEST=[]

if(DATASET!='ELM'):
    VAL=np.load(f'./VALIDATION_EXA/{DATASET}.npy')
    adj=0
else:
    VAL=np.genfromtxt(f'./VALIDATION_EXA/{DATASET}.csv')
    adj=1
for i in range(1,35):  
#for i in range(0,34):
    

    GT.append(np.genfromtxt(f'./SCORE_EXA/GT/{i}.csv'))
    TEST.append(np.genfromtxt(f'./SCORE_EXA/{DATASET}/{i-1}.csv'))

GT=np.array(GT)
TEST=np.array(TEST)


C:\Users\gioel\AppData\Local\Temp\ipykernel_672\4086280527.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  GT=np.array(GT)
C:\Users\gioel\AppData\Local\Temp\ipykernel_672\4086280527.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  TEST=np.array(TEST)


In [9]:


import csv
import pickle

header=['th_factor','method','value','F1','precision','recall','TP','TN','FP','FN']

with open(f'{DATASET}.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerow(header)
    thresholding_factor=[0.5,1,1.5,2]
    for t in thresholding_factor:

        score=VAL
        
        #IQR
        q1, q3 = np.percentile(score, 25), np.percentile(score, 75)
        iqr = q3 - q1
        IQR = q3 + t* iqr

        #MAD
        median = np.median(score)
        mad = 1.4826 * np.median(np.abs(score - median))
        MAD = median + t * mad

        #STD
        mean, std = np.mean(score), np.std(score)
        STD = mean + t * std

        method=[IQR,MAD,STD]
        
        for g in range(len(method)):
            TP=0
            TN=0
            FN=0
            FP=0

            for i in range(0,34):

                                '''if(DATASET=='ELM'):
                                    if(method[g]>0.5):
                                        out=TEST[i]>=0.5
                                    else:
                                        out=TEST[i]>method[g]'''

                                out=TEST[i]>method[g]

                                le=len(TEST[i])

                                true_= GT[i][39:le+39]>=1
                                #out=[False]*(len(true_)-len(out)+adj)+list(out)
                                #print(out)
                                prediction_ =out ==1
                              

                                TP = TP+(true_ & prediction_).sum()   
                                TN = TN+(~true_ & ~prediction_).sum()  
                                FP = FP+(~true_ & prediction_).sum()    
                                FN = FN+(true_ & ~prediction_).sum()        

            PREC=TP / (TP + FP)
            REC = TP/ (TP+FN)
            f1=2 * PREC * REC/(PREC + REC)
            m=['IQR','MAD','STD']
            row=[t,m[g],method[g],f1,PREC,REC,TP,TN,FP,FN]
            writer.writerow(row)

    #MAX
    for g in range(len(method)):
            TP=0
            TN=0
            FN=0
            FP=0
            th=np.max(VAL)
            for i in range(0,34):

                                if(DATASET=='ELM'):
                                    if(th>0.5):
                                        out=TEST[i]>=0.5
                                    else:
                                        out=TEST[i]>th
                                else:
                                        out=TEST[i]>th

                                le=len(TEST[i])
                                
                                #true_= GT[i]>=1
                                #out=[False]*(len(true_)-len(out)+adj)+list(out)
                                true_= GT[i][39:le+39]>=1
                                prediction_ =out ==1
                                
                                TP = TP+(true_ & prediction_).sum()   
                                TN = TN+(~true_ & ~prediction_).sum()  
                                FP = FP+(~true_ & prediction_).sum()    
                                FN = FN+(true_ & ~prediction_).sum()        

            PREC=TP / (TP + FP)
            REC = TP/ (TP+FN)
            f1=2 * PREC * REC/(PREC + REC)
    row=['','MAX',th,f1,PREC,REC,TP,TN,FP,FN]
    writer.writerow(row)
